In [43]:
import sqlite3
import os
from faker import Faker
import random
import asyncio



In [44]:

# Ensure the 'data' folder exists
os.makedirs('data', exist_ok=True)

# Define the database path
db_path = os.path.join('data', 'synthetic_socialcare.db')

# Connect to the SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()




### Defined SQLite database schema for social care tables: clients, assessments, services, outcomes.

In [45]:

# Connect to database (creates file if not exists)
conn = sqlite3.connect("data/synthetic_socialcare.db")
cursor = conn.cursor()

# Drop old tables (if running again for testing)
cursor.execute("DROP TABLE IF EXISTS clients")
cursor.execute("DROP TABLE IF EXISTS assessments")
cursor.execute("DROP TABLE IF EXISTS services")
cursor.execute("DROP TABLE IF EXISTS outcomes")

# Create Clients table
cursor.execute("""
CREATE TABLE clients (
    client_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    gender TEXT,
    postcode TEXT
)
""")

# Create Assessments table
cursor.execute("""
CREATE TABLE assessments (
    assessment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    client_id INTEGER,
    assessment_date TEXT,
    assessment_type TEXT,
    assessor TEXT,
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
)
""")

# Create Services table
cursor.execute("""
CREATE TABLE services (
    service_id INTEGER PRIMARY KEY AUTOINCREMENT,
    client_id INTEGER,
    service_name TEXT,
    start_date TEXT,
    end_date TEXT,
    provider TEXT,
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
)
""")

# Create Outcomes table
cursor.execute("""
CREATE TABLE outcomes (
    outcome_id INTEGER PRIMARY KEY AUTOINCREMENT,
    client_id INTEGER,
    outcome_date TEXT,
    outcome_type TEXT,
    outcome_value TEXT,
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
)
""")

# Save changes
conn.commit()
print("Tables created successfully.")

conn.close()


Tables created successfully.


### Populated tables with synthetic data using Faker.

In [46]:

# Setup
fake = Faker("en_GB")  # UK-style data
conn = sqlite3.connect("data/synthetic_socialcare.db")
cursor = conn.cursor()

# Number of synthetic clients
N_CLIENTS = 50

for client_id in range(1, N_CLIENTS + 1):
    # ---------- CLIENT ----------
    name = fake.name()
    age = random.randint(18, 95)
    gender = random.choice(["Male", "Female"])
    postcode = fake.postcode()
    
    cursor.execute("""
    INSERT INTO clients (client_id, name, age, gender, postcode)
    VALUES (?, ?, ?, ?, ?)
    """, (client_id, name, age, gender, postcode))
    
    # ---------- ASSESSMENTS ----------
    for _ in range(random.randint(1, 3)):  # 1–3 assessments per client
        cursor.execute("""
        INSERT INTO assessments (client_id, assessment_date, assessment_type, assessor)
        VALUES (?, ?, ?, ?)
        """, (
            client_id,
            fake.date_between(start_date="-2y", end_date="today"),
            random.choice(["Care Act", "Financial", "Risk"]),
            fake.name()
        ))
    
    # ---------- SERVICES ----------
    for _ in range(random.randint(1, 2)):  # 1–2 services per client
        start_date = fake.date_between(start_date="-2y", end_date="-1m")
        end_date = fake.date_between(start_date=start_date, end_date="today")
        cursor.execute("""
        INSERT INTO services (client_id, service_name, start_date, end_date, provider)
        VALUES (?, ?, ?, ?, ?)
        """, (
            client_id,
            random.choice(["Home Care", "Residential Care", "Day Centre", "Direct Payment"]),
            start_date,
            end_date,
            random.choice(["Local Authority", "Private Agency", "Charity"])
        ))
    
    # ---------- OUTCOMES ----------
    for _ in range(random.randint(1, 2)):  # 1–2 outcomes per client
        cursor.execute("""
        INSERT INTO outcomes (client_id, outcome_date, outcome_type, outcome_value)
        VALUES (?, ?, ?, ?)
        """, (
            client_id,
            fake.date_between(start_date="-1y", end_date="today"),
            random.choice(["Independence", "Wellbeing", "Safety", "Financial"]),
            random.choice(["Improved", "Maintained", "Declined"])
        ))

# Save changes
conn.commit()
print("Synthetic data inserted with consistent client_id.")

conn.close()


Synthetic data inserted with consistent client_id.


C:\Users\Admin\AppData\Local\Temp\ipykernel_33828\2531899141.py:23: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""
C:\Users\Admin\AppData\Local\Temp\ipykernel_33828\2531899141.py:37: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""
C:\Users\Admin\AppData\Local\Temp\ipykernel_33828\2531899141.py:50: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""


In [47]:

conn = sqlite3.connect("data/synthetic_socialcare.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

conn.close()


[('sqlite_sequence',), ('clients',), ('assessments',), ('services',), ('outcomes',)]


In [ ]:
# Example schema dictionary
schema = {
    "clients": ["client_id", "name", "age", "gender", "postcode"],
    "assessments": ["assessment_id", "client_id", "assessment_date", "outcome"],
    "services": ["service_id", "client_id", "service_name", "start_date", "end_date"],
    "outcomes": ["client_id", "outcome_date", "outcome_type", "outcome_value"]
}


## Build AI SQL Generator

### Setup Environment and API keys

In [68]:
from openai import OpenAI
from dotenv import load_dotenv
from agents import Agent, Runner, trace
import os
import asyncio

In [69]:
load_dotenv(override=True)

True

In [70]:
openaikey = os.getenv("OPENAI_API_KEY")

if openaikey:
    print(f"The key is found and starts with: {openaikey[:8]}")
else:
    print(f"Key not found")

The key is found and starts with: sk-proj-


#### SQL execution function to run read-only queries on SQLite DB.

In [84]:
def execute_sql(query: str):
    """Run a read-only SQL query on the SQLite DB."""
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        colnames = [desc[0] for desc in cursor.description]
    return [dict(zip(colnames, row)) for row in rows]

#### Configure Agent instance (QueryExecutor) with gpt-4o-mini model.

In [ ]:
instruction1 = f"""
You are a SQL assistant for adult social care data.
Always generate SELECT-only SQL for the SQLite schema provided.
Never use DROP, INSERT, UPDATE, DELETE, ALTER, ATTACH, or PRAGMA.
Return strictly JSON with keys:
- sql: the SQL query string
- explanation: a short (1-2 sentence) explanation of the query logic
Schema reference: {schema}
Make sure the output is a single valid JSON object, nothing else.
"""


In [86]:
Agent1 = Agent(
    name = 'QueryExecutor',
    instructions = instruction1,
    model = "gpt-4o-mini"
)

In [87]:
Agent1

Agent(name='QueryExecutor', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions="\nYou are a SQL assistant for adult social care data.\nAlways generate SELECT-only SQL for the SQLite schema provided.\nNever use DROP, INSERT, UPDATE, DELETE, ALTER, ATTACH, or PRAGMA.\nReturn strictly JSON with keys:\n- sql: the SQL query string\n- explanation: a short (1-2 sentence) explanation of the query logic\nSchema reference: {'clients': ['client_id', 'name', 'age', 'gender', 'postcode'], 'assessments': ['assessment_id', 'client_id', 'assessment_date', 'outcome'], 'services': ['service_id', 'client_id', 'service_name', 'start_date', 'end_date'], 'outcomes': ['client_id', 'outcome_date', 'outcome_type', 'outcome_value']}\nMake sure the output is a single valid JSON object, nothing else.\n", prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_too

#### LLM-generated SQL queries, executions and results

In [99]:
result = await Runner.run(Agent1, "List the adults aged over 20 years with their service outcome")

print(result.final_output)

{
  "sql": "SELECT c.client_id, c.name, c.age, o.outcome_type, o.outcome_value FROM clients c LEFT JOIN outcomes o ON c.client_id = o.client_id WHERE c.age > 20;",
  "explanation": "This query selects clients over 20 years old and retrieves their names, ages, and associated outcomes by performing a LEFT JOIN between the clients and outcomes tables."
}


In [100]:
print("Raw LLM output:", repr(result.final_output))


Raw LLM output: '{\n  "sql": "SELECT c.client_id, c.name, c.age, o.outcome_type, o.outcome_value FROM clients c LEFT JOIN outcomes o ON c.client_id = o.client_id WHERE c.age > 20;",\n  "explanation": "This query selects clients over 20 years old and retrieves their names, ages, and associated outcomes by performing a LEFT JOIN between the clients and outcomes tables."\n}'


In [101]:
import json

clean_output = result.final_output.strip().strip('"')
parsed_result = json.loads(clean_output)

In [102]:
# Now you can access keys
sql_query = parsed_result["sql"]
explanation = parsed_result["explanation"]

In [103]:
# 🔹 Execute SQL
query_results = execute_sql(sql_query)

# Now you have BOTH:
print("🔹 Generated SQL:", sql_query)
#print("🔹 Agent Explanation:", result.final_output["explanation"])
print("🔹 Executed Results:", query_results)

🔹 Generated SQL: SELECT c.client_id, c.name, c.age, o.outcome_type, o.outcome_value FROM clients c LEFT JOIN outcomes o ON c.client_id = o.client_id WHERE c.age > 20;
🔹 Executed Results: [{'client_id': 1, 'name': 'Miss Laura Finch', 'age': 55, 'outcome_type': 'Financial', 'outcome_value': 'Maintained'}, {'client_id': 1, 'name': 'Miss Laura Finch', 'age': 55, 'outcome_type': 'Safety', 'outcome_value': 'Improved'}, {'client_id': 2, 'name': 'Ms Amy Harrison', 'age': 33, 'outcome_type': 'Wellbeing', 'outcome_value': 'Declined'}, {'client_id': 2, 'name': 'Ms Amy Harrison', 'age': 33, 'outcome_type': 'Wellbeing', 'outcome_value': 'Improved'}, {'client_id': 3, 'name': 'Dr Shaun Thomas', 'age': 95, 'outcome_type': 'Financial', 'outcome_value': 'Declined'}, {'client_id': 4, 'name': 'Dr Jeremy Richards', 'age': 85, 'outcome_type': 'Financial', 'outcome_value': 'Improved'}, {'client_id': 5, 'name': 'Dr Natalie Holt', 'age': 61, 'outcome_type': 'Financial', 'outcome_value': 'Declined'}, {'client_i

In [96]:
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    tables = ["clients", "services", "outcomes"]
    for t in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {t}")
        print(f"{t} rows:", cursor.fetchone()[0])


clients rows: 50
services rows: 78
outcomes rows: 74
